In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as pltc
import datetime as dt
from scipy import stats
from sklearn import preprocessing
import matplotlib as mpl
from scipy.stats import skew
import geopandas
import seaborn as sns

## Plotting parameters
mpl.rcParams['pdf.fonttype'] = 42

## Cubic feet to cubic meters conversion factor
cfs_2_cms = 0.0283168466

In [2]:
def readModelFile(i):
    try:
        df = pd.read_csv('../data/pnwNP_modeledData_NWM3/'+str(pnwNP['gage'][i])+".csv")
        df['time'] = pd.to_datetime(df['time'])
       
    ### Deliniate time periods
        df['climate_year'] = df.time.dt.year.where(df.time.dt.month < 4, df.time.dt.year + 1)
        df.loc[(df.time.dt.month > 7) & (df.time.dt.month < 10), "flow_season"] = "late_summer"
        df.loc[(df.time.dt.month > 2) & (df.time.dt.month < 6), "season"] = "spring"
        df.loc[(df.time.dt.month > 5) & (df.time.dt.month < 9), "season"] = "summer"
        df.loc[(df.time.dt.month > 8) & (df.time.dt.month < 12), "season"] = "fall"
        df.loc[(df.time.dt.month > 11), "season"]  = "winter"
        df.loc[(df.time.dt.month < 3), "season"] = "winter"
        df['month'] = df.time.dt.month
        df["DOY"] = df.time.dt.day_of_year
        
        # df = df.drop(['Unnamed: 0.1'],axis=1)
        df = df[df["streamflow_NWIS"].notna()].reset_index(drop=True)
        return df

    except:
        pass

In [88]:
def calc7Q10(df):
    import warnings
    warnings.filterwarnings("ignore")
    '''
    Read in data files for gages and calculate frequency statistics.
    '''
    
    stat = "7Q10"
    columns = ["gage","VIC_"+stat,"PRMS_"+stat,"NWM2d0_"+stat,"NWM2d1_"+stat,"NWM3"+stat,"NWIS_"+stat]
    out = []

    for column in df[["streamflow_VIC","streamflow_PRMS",'streamflow_NWM2d0','streamflow_NWM2d1','streamflow_NWM3','streamflow_NWIS']]:
       
    
        #### Round columns to nearest 0.1 cfs, therefore anything less than 0.05 cfs is q=0
        epsil = 1e-4
        if column == "streamflow_NWIS":
            df[column] = df[column]/cfs_2_cms
            df[column] = np.round(df[column],1)
            df[column] = (df[column]*cfs_2_cms)+epsil
        else:
            pass
        
        df.loc[~df[column].isnull(),column] = df[column]+epsil
        
        
        ## Calculate 7-day rolling average 
        df['7Q10'] = df[column].rolling(window=7).mean()


        lf7q10 = df[['climate_year','7Q10']]
        lf7q10 = lf7q10.groupby(['climate_year']).min()
        lf7q10 = lf7q10[df['7Q10'].isnull().groupby(df['climate_year']).sum() <= 36.6] ## makes sure that gage has at least 90% of year with data

        lf7q10["RI"] = (1+len(lf7q10))/lf7q10['7Q10'].rank(method = "min")
        lf7q10["ExcProb"] = 1/lf7q10["RI"]


        #Measures of the distribution
        Xbar = np.mean(np.log10(lf7q10['7Q10']))
        S = np.std(np.log10(lf7q10['7Q10']))
        g = skew(np.log10(lf7q10['7Q10']))

        ## Calculate Pearson type 3 coeff 
        lf7q10["z"] = 4.91 * ((1 / lf7q10["RI"])**0.14 -(1 - 1 / lf7q10["RI"])**0.14)
        lf7q10["K"] = (2/g) * ((((lf7q10["z"] - (g/6))*(g/6)+1)**3)-1)
        lf7q10["Qfit"] = 10**(Xbar + (lf7q10["K"] * S))

        ## Calculate 7q#
        RetIntCalc = 10
        z = 4.91 * ((1 / RetIntCalc)**0.14 -(1 - 1 / RetIntCalc)**0.14)
        K = (2/g) * ((((z - (g/6))*(g/6)+1)**3)-1)
        calc7q10 = 10**(Xbar + (K * S))
        out.extend([calc7q10])

    out.insert(0,int(df["gage"][0]))
    out = pd.DataFrame(out).transpose()
    out.columns = columns
    return out


In [78]:
# pnwNP = pd.read_csv("../data/pnwNP_StatsLocations_HydroClass.csv")
pnwNP = kgeGages = pd.read_csv("../data/KGE_gages.csv")



i=2
df = readModelFile(i)
out = calc7Q10(df)
out


,gage,VIC_7Q10,PRMS_7Q10,NWM2d0_7Q10,NWM2d1_7Q10,NWM37Q10,NWIS_7Q10
0,12090400.0,0.103199,0.012687,0.075121,0.136008,0.056232,NaN


In [84]:
df = pd.read_csv("../data/DELETE_7Q10_testData.csv")
df['time'] = pd.to_datetime(df['time'])

### Deliniate time periods
df['climate_year'] = df.time.dt.year.where(df.time.dt.month < 4, df.time.dt.year + 1)
df.loc[(df.time.dt.month > 7) & (df.time.dt.month < 10), "flow_season"] = "late_summer"
df.loc[(df.time.dt.month > 2) & (df.time.dt.month < 6), "season"] = "spring"
df.loc[(df.time.dt.month > 5) & (df.time.dt.month < 9), "season"] = "summer"
df.loc[(df.time.dt.month > 8) & (df.time.dt.month < 12), "season"] = "fall"
df.loc[(df.time.dt.month > 11), "season"]  = "winter"
df.loc[(df.time.dt.month < 3), "season"] = "winter"
df['month'] = df.time.dt.month
df["DOY"] = df.time.dt.day_of_year

# df = df.drop(['Unnamed: 0.1'],axis=1)
df = df[df["streamflow_NWIS"].notna()].reset_index(drop=True)
df

,Unnamed: 0.1,Unnamed: 0,streamflow_VIC,time,streamflow_PRMS,streamflow_NWM2d0,streamflow_NWM2d1,streamflow_NWIS,gage,streamflow_NWM3,climate_year,flow_season,season,month,DOY
0,10623,10623.0,NaN,1979-02-01,0.397391,0.000397,0.003974,0.397391,12090400.0,0.397391,1979,NaN,winter,2,32
1,10624,10624.0,NaN,1979-02-02,0.382500,0.000382,0.003825,0.382500,12090400.0,0.382500,1979,NaN,winter,2,33
2,10625,10625.0,NaN,1979-02-03,0.369583,0.000370,0.003696,0.369583,12090400.0,0.369583,1979,NaN,winter,2,34
3,10626,10626.0,NaN,1979-02-04,0.414583,0.000415,0.004146,0.414583,12090400.0,0.414583,1979,NaN,winter,2,35
4,10627,10627.0,NaN,1979-02-05,0.827917,0.000828,0.008279,0.827917,12090400.0,0.827917,1979,NaN,winter,2,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16067,26690,NaN,0.438750,2023-01-28,0.438750,0.000439,0.004387,0.438750,NaN,0.438750,2023,NaN,winter,1,28
16068,26691,NaN,0.411667,2023-01-29,0.411667,0.000412,0.004117,0.411667,NaN,0.411667,2023,NaN,winter,1,29
16069,26692,NaN,0.400000,2023-01-30,0.400000,0.000400,0.004000,0.400000,NaN,0.400000,2023,NaN,winter,1,30
16070,26693,NaN,0.408750,2023-01-31,0.408750,0.000409,0.004087,0.408750,NaN,0.408750,2023,NaN,winter,1,31


In [87]:
calc7Q10(df)

,gage,VIC_7Q10,PRMS_7Q10,NWM2d0_7Q10,NWM2d1_7Q10,NWM37Q10,NWIS_7Q10
0,12090400.0,0.05598,0.000022,0.000158,0.000663,0.05598,0.056203


In [89]:
from tqdm import tqdm
pnwNP = kgeGages = pd.read_csv("../data/KGE_gages.csv")

output = pd.DataFrame([])
for i in tqdm(range(len(pnwNP))):
    df = readModelFile(i)
    out = calc7Q10(df)
    output = pd.concat([output, out], ignore_index=True)
    del(out,df)

100%|████████████████████████████████████████████████████████████████████████████████| 126/126 [00:13<00:00,  9.18it/s]


In [90]:
output

,gage,VIC_7Q10,PRMS_7Q10,NWM2d0_7Q10,NWM2d1_7Q10,NWM37Q10,NWIS_7Q10
0,10393500.0,4.145477,0.981530,1.573867,0.726650,0.335530,0.082941
1,12090365.0,0.097427,0.008941,0.099208,0.142014,0.058580,NaN
2,12090400.0,0.103297,0.012781,0.075220,0.136110,0.056335,0.000130
3,12090500.0,0.637010,0.154044,0.873230,1.495777,0.628946,0.000301
4,12091200.0,0.108191,0.016739,NaN,NaN,NaN,0.024148
...,...,...,...,...,...,...,...
121,14337800.0,0.576430,0.147030,0.195557,0.197107,0.099345,0.035393
122,14337830.0,0.755319,0.198570,0.238833,0.231271,0.112056,0.026554
123,14338000.0,0.909110,0.274085,0.244895,0.274845,0.139488,0.020420
124,14353000.0,0.077251,0.025980,0.031380,NaN,0.091127,0.038278


In [94]:
output[output.NWIS_7Q10<1e-4]

,gage,VIC_7Q10,PRMS_7Q10,NWM2d0_7Q10,NWM2d1_7Q10,NWM37Q10,NWIS_7Q10
8,12115700.0,0.029687,0.025239,NaN,0.030127,0.015076,0.000089
18,12210900.0,0.123521,0.046660,0.000610,0.009342,0.058449,0.000095
27,12447383.0,0.851545,4.810534,1.471999,1.298688,3.638435,0.000070
36,13016315.0,0.576947,0.501293,0.537825,0.767542,1.297805,0.000034
41,13058000.0,3.061744,2.077922,1.353080,0.995293,1.524955,0.000037
46,130875048.0,100.484862,115.527723,100.709083,188.424118,202.053674,0.000036
47,130875049.0,100.484862,115.527723,100.709083,188.424118,202.053674,0.000041
48,13087505.0,107.085861,125.216178,97.565766,189.486552,204.499820,0.000019
51,13112000.0,1.411678,0.308192,NaN,0.269517,0.367589,0.000051
53,13132500.0,4.499519,1.237074,5.094298,2.471672,3.488325,0.000073


NameError: name 'df' is not defined

In [41]:
df

,Unnamed: 0.1,Unnamed: 0,streamflow_VIC,time,streamflow_PRMS,streamflow_NWM2d0,streamflow_NWM2d1,streamflow_NWIS,gage,streamflow_NWM3,climate_year,flow_season,season,month,DOY,7Q10,new
0,0,0.0,0.257712,1950-01-01,3.853442,NaN,NaN,0.006663,13217500.0,NaN,1950,NaN,winter,1,1,NaN,NaN
1,1,1.0,0.434619,1950-01-02,6.500389,NaN,NaN,0.006663,13217500.0,NaN,1950,NaN,winter,1,2,NaN,NaN
2,2,2.0,0.480279,1950-01-03,7.042893,NaN,NaN,0.006663,13217500.0,NaN,1950,NaN,winter,1,3,NaN,NaN
3,3,3.0,0.487879,1950-01-04,6.931520,NaN,NaN,0.006663,13217500.0,NaN,1950,NaN,winter,1,4,NaN,NaN
4,4,4.0,0.488187,1950-01-05,6.697767,NaN,NaN,0.006663,13217500.0,NaN,1950,NaN,winter,1,5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24701,24740,24740.0,NaN,2017-09-26,NaN,1.961000,4.093500,2.558011,13217500.0,2.721833,2018,late_summer,fall,9,269,3.079041,NaN
24702,24741,24741.0,NaN,2017-09-27,NaN,1.961000,4.076000,2.524031,13217500.0,2.714333,2018,late_summer,fall,9,270,2.873137,NaN
24703,24742,24742.0,NaN,2017-09-28,NaN,1.959333,4.058500,2.518368,13217500.0,2.706833,2018,late_summer,fall,9,271,2.775646,NaN
24704,24743,24743.0,NaN,2017-09-29,NaN,1.951000,4.041833,2.475892,13217500.0,2.699750,2018,late_summer,fall,9,272,2.720631,NaN


In [42]:
column = "streamflow_NWM3"
~df[column].isnull()+epsil


0        0.001
1        0.001
2        0.001
3        0.001
4        0.001
         ...  
24701    1.001
24702    1.001
24703    1.001
24704    1.001
24705    1.001
Name: streamflow_NWM3, Length: 24706, dtype: float64

In [52]:
df[~df[column].isnull()][column]

10584    2.044478
10585    1.971417
10586    1.931417
10587    1.931000
10588    1.931000
           ...   
24701    2.721833
24702    2.714333
24703    2.706833
24704    2.699750
24705    2.691000
Name: streamflow_NWM3, Length: 14122, dtype: float64

In [53]:
df.loc[~df[column].isnull(),column] = df[column]+epsil

In [54]:
df[~df[column].isnull()][column]

10584    2.045478
10585    1.972417
10586    1.932417
10587    1.932000
10588    1.932000
           ...   
24701    2.722833
24702    2.715333
24703    2.707833
24704    2.700750
24705    2.692000
Name: streamflow_NWM3, Length: 14122, dtype: float64

In [57]:
pnwNP.gage == 12090400

0      False
1      False
2      False
3      False
4      False
       ...  
462    False
463    False
464    False
465    False
466    False
Name: gage, Length: 467, dtype: bool